In [15]:
import MySQLdb
import MySQLdb.cursors
import requests
import pandas as pd
import numpy as np

cnx = MySQLdb.connect(host = 'localhost',
                     database = 'cb-dev',
                     user = 'root',
                     password = 'root', 
                     compress = 1,
                     cursorclass=MySQLdb.cursors.DictCursor)

cursor = cnx.cursor()

query = """select jobs.title, jobs.job_role,
            group_concat(distinct(select name from industries where id = industry_maps.industry_id)) as ind,
            group_concat(distinct(select name from competencies where id = competency_maps.competency_id)) as comp

            from jobs
            inner join industry_maps on industry_maps.industry_mappable_id = jobs.id
            inner join competency_maps on competency_maps.competency_mappable_id = jobs.id
            group by industry_maps.industry_mappable_id, competency_maps.competency_mappable_id""" 

cursor.execute(query)

tab = cursor.fetchall()

df = pd.DataFrame(list(tab))
df.head()


,comp,ind,job_role,title
0,"Entrepreneurial skills,Enterprise architecture...",Consulting,"<div class=""block fix-text"" itemprop=""descript...",Head of Service (Healthcare)
1,Administration skills,Construction,"<div class=""block fix-text"" itemprop=""descript...","Financial Controller, Maternity Cover"
2,Strategic expertise,Information Technology,"<div class=""block fix-text"" itemprop=""descript...",Education Safeguarding Officer
3,"Administration skills,Business intelligence,Cu...",Consulting,"<div class=""block fix-text"" itemprop=""descript...",National Media Manager
4,"Business development and sales,Change leadersh...",Consulting,"<div class=""block fix-text"" itemprop=""descript...",Director of Innovation Consultancy


In [16]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [19]:
df.isnull().any()

comp        False
ind         False
job_role    False
title       False
dtype: bool

In [20]:
df['comp'] = df['comp'].map(lambda x: x.split(","))
df['comp'].head()

0    [Entrepreneurial skills, Enterprise architectu...
1                              [Administration skills]
2                                [Strategic expertise]
3    [Administration skills, Business intelligence,...
4    [Business development and sales, Change leader...
Name: comp, dtype: object

In [21]:
df['ind'] = df['ind'].map(lambda x: x.split(","))

In [22]:
for index, row in df.iterrows():
    row['comp'] = [x.lower().replace(' ','') for x in row['comp']]
    row['ind'] = [x.lower().replace(' ','') for x in row['ind']]
    #row['ind'] = ''.join(row['ind']).lower()

In [24]:
df['Key_words'] = ""

for index, row in df.iterrows():
    plot = row['job_role']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(['job_role'],axis = 1, inplace = True)

In [25]:
df.head()

,comp,ind,title,Key_words
0,"[entrepreneurialskills, enterprisearchitecture...",[consulting],Head of Service (Healthcare),"[majority, operations, accompany, full, time, ..."
1,[administrationskills],[construction],"Financial Controller, Maternity Cover","[000, per, annum, .</, p, wednesday, 27, septe..."
2,[strategicexpertise],[informationtechnology],Education Safeguarding Officer,"[south, east, medium, england, uphold, standar..."
3,"[administrationskills, businessintelligence, c...",[consulting],National Media Manager,"[right, messages, excellent, team, leader, rol..."
4,"[businessdevelopmentandsales, changeleadership...",[consulting],Director of Innovation Consultancy,"[three, posts, public, thriving, societies, au..."


In [26]:
df.set_index('title', inplace = True)
df.head()

,comp,ind,Key_words
title,,,
Head of Service (Healthcare),"[entrepreneurialskills, enterprisearchitecture...",[consulting],"[majority, operations, accompany, full, time, ..."
"Financial Controller, Maternity Cover",[administrationskills],[construction],"[000, per, annum, .</, p, wednesday, 27, septe..."
Education Safeguarding Officer,[strategicexpertise],[informationtechnology],"[south, east, medium, england, uphold, standar..."
National Media Manager,"[administrationskills, businessintelligence, c...",[consulting],"[right, messages, excellent, team, leader, rol..."
Director of Innovation Consultancy,"[businessdevelopmentandsales, changeleadership...",[consulting],"[three, posts, public, thriving, societies, au..."


In [27]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'comp' or col != 'ind':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop([col for col in df.columns if col!= 'bag_of_words'],axis = 1, inplace = True)

In [28]:
df.head()

,bag_of_words
title,
Head of Service (Healthcare),entrepreneurialskills enterprisearchitecture s...
"Financial Controller, Maternity Cover",administrationskills construction 000 per annu...
Education Safeguarding Officer,strategicexpertise informationtechnology south...
National Media Manager,administrationskills businessintelligence cust...
Director of Innovation Consultancy,businessdevelopmentandsales changeleadership c...


In [29]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0             Head of Service (Healthcare)
1    Financial Controller, Maternity Cover
2           Education Safeguarding Officer
3                   National Media Manager
4       Director of Innovation Consultancy
Name: title, dtype: object

In [30]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[ 1.        ,  0.2121394 ,  0.14689964, ...,  0.05197192,
         0.0244998 ,  0.        ],
       [ 0.2121394 ,  1.        ,  0.13436308, ...,  0.04859295,
         0.03436041,  0.        ],
       [ 0.14689964,  0.13436308,  1.        , ...,  0.05266807,
         0.03724195,  0.        ],
       ..., 
       [ 0.05197192,  0.04859295,  0.05266807, ...,  1.        ,
         0.20203051,  0.15430335],
       [ 0.0244998 ,  0.03436041,  0.03724195, ...,  0.20203051,
         1.        ,  0.10910895],
       [ 0.        ,  0.        ,  0.        , ...,  0.15430335,
         0.10910895,  1.        ]])

In [45]:
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]
    #print(idx)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    #print(score_series)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [48]:
recommendations('Account Collector')

['Accounting Clerks',
 'Account Management, Manager',
 'Accounting Vice President',
 'Account Collector-2203',
 'Testing Job',
 'Claims Adjuster',
 'CCO (Chief Contracting Officer)-1',
 'CCO (Chief Contracting Officer)',
 'Audit Supervisor',
 'Project Manager Level 5']